In [3]:
import ipywidgets as widgets
from IPython.display import display #mostrar controles
import pandas as pd
import io
import os
import matplotlib.pyplot as plt
import numpy as np
from docplex.mp.model import Model
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')# ignorar warnings
import panel as pn
pn.extension()
# Cambiar de path
current_directory=os.getcwd()
os.chdir(os.path.split(current_directory)[0])

# librerías locales para el proyecto
from modulos.preprocesamiento.limpieza_datos import escoger_variables,limpiar_dataframe,plot_raw_map
from modulos.preprocesamiento.encoding import escalar_variables
from modulos.crear_clusters.plotear_clusters import entrenar_cluster,plot_clusters
from modulos.optimizacion.ruteo_vehiculos import ruteo_dinamico,plot_mejor_ruta,presentar_ruta
from modulos.directorio.filedialog import guardar_ruta
#  ----------------------------------------------------------------------------
#        INSTRUCCIONES DE PANTALLAS
# ----------------------------------------------------------------------------
# INSTRUCCIÓN PANTALLA INICIO
INSTRUCCIONES_DE_PESTANA_INICIO=widgets.HTML('''
    <h2>Proyeyecto de Python Fundamentals</h2>
    <h3>Distribución: cluster de puntos de ventas y enrutamiento de vehículos</h3>
    <p>La pestaña Inicio es para cargar los datos a evaluar de distribución</p>
    <p>Se acota que el proyecto brinda las rutas para cada cluster, no a nivel global</p>
    <p>Se debe realizar el siguiente procedimiento</p>
    Pasos:
    <ul>
    <li>Cargar el archivo en formato .xlsx o .csv (máximo 10MB)</li>
    <li>Hacer click en Mostrar los datos para verificar que sean los subidos</li>
    <li>Los datos mostrados contemplan 10 primeros registros e información de variables</li>
    <li>Ir a las siguientes pestañas para ver los clusters y las rutas asignadas, se debe ir en orden</li>
  </ul>
''')
# INSTRUCCIÓN PANTALLA PLOT RAWMAP
INSTRUCCIONES_DE_RAWMAP=widgets.HTML('''
    <h3>Distribución: Plot de puntos de ventas del día </h3>
    <h4>En esta sección se muestra en un mapa los puntos de venta actuales</h4>
    <p>Se debe seleccionar el día y presionar el boton para plotear el mapa actual sin clusters</p>
    <p>Después de ver los puntos, se debe dirigir a la pestaña de clusters</p>
''')

# INSTRUCCIÓN PANTALLA CLUSTER
INSTRUCCIONES_DE_CLUSTERS=widgets.HTML('''
    <h3>Distribución: Clustering por medio de KMeans y GaussianMixture </h3>
    <h4>En esta sección se escoge también el dia y el modelo del cluster a emplear</h4>
    <p>Si bien un dia y tipo de clustering van en pareja, este proceso es estocástico, por lo que 
    contar si se vuelve a presionar el botón botará otras agrupacoines</p>
    <p>Después de realizar el clustering, proceder a la pestaña de optimización</p>
''')

# INSTRUCCIÓN PANTALLA OPTIMIZACION
INSTRUCCIONES_DE_OPTIMIZACION=widgets.HTML('''
    <h3>Distribución: Optimización por medio de Cplex de IBM</h3>
    <h4>En esta sección se traza la mejor ruta para cada cluster obtenido en el pestaña previa</h4>
    <p>Este versión de Cplex de prueba solo permite obtener mejor enrutamiento para una cantidad
    determinada de puntos en un cluster, por lo que se realiza la optimización por cada cluster</p>
    <p>Se presionar el boton para empezar con la generación de mejores rutas</p>
''')

# INSTRUCCIÓN PANTALLA RESULTADOS
INSTRUCCIONES_DE_OPTIMIZACION=widgets.HTML('''
    <h3>Distribución: </h3>
    <h4>En esta sección se traza la mejor ruta para cada cluster obtenido en el pestaña previa</h4>
    <p>Este versión de Cplex de prueba solo permite obtener mejor enrutamiento para una cantidad
    determinada de puntos en un cluster, por lo que se realiza la optimización por cada cluster</p>
    <p>Se presionar el boton para empezar con la generación de mejores rutas</p>
''')
#  ----------------------------------------------------------------------------
#        ETIQUETAS INTERACTIVAS
# ----------------------------------------------------------------------------
ETIQUETA_DIA_CLUSTER=widgets.HTML()
ETIQUETA_RESULTADOS_OPTIMIZACION=widgets.HTML()

#  ----------------------------------------------------------------------------
#        BOTONES
# ----------------------------------------------------------------------------
# definir botones de carga
UPLOAD=widgets.FileUpload(accepts='',
                          multiple=False,
                          description='Subir archivo en .xlsx o .csv',
                          layout=widgets.Layout(width='220px'))

BOTON_MOSTRAR_DATOS=widgets.Button(
                    description='Mostrar datos',
                    disabled=False,
                    button_style='info',
                    tooltip='Mostrar',
                    icon='fa-spinner')

BOTON_PLOTEAR_MAPA=widgets.Button(
                    description='Plotear puntos de venta',
                    disabled=False,
                    button_style='warning',
                    tooltip='Mostrar',
                    icon='fa-map',
                    layout=widgets.Layout(width='200px'))

BOTON_PLOTEAR_CLUSTER=widgets.Button(
                    description='Plotear clusters de los puntos de venta',
                    disabled=False,
                    button_style='success',
                    tooltip='Mostrar',
                    icon='fa-taxi',
                    layout=widgets.Layout(width='280px'))

BOTON_PLOTEAR_RUTAS=widgets.Button(
                    description='Plotear enrutamiento por cluster',
                    disabled=False,
                    button_style='danger',
                    tooltip='Mostrar',
                    icon='fa-truck',
                    layout=widgets.Layout(width='280px'))

BOTON_GUARDAR_RESULTADOS=widgets.Button(
                    description='Exportar rutas',
                    disabled=False,
                    button_style='',
                    tooltip='Mostrar',
                    icon='fa-download',
                    layout=widgets.Layout(width='180px'))

#  ----------------------------------------------------------------------------
#        SELECCIÓN SIMPLE
# ----------------------------------------------------------------------------
# definir dia a seleccionar del dataframe
SELECCION_DIA=widgets.Select(
                    options=['lunes', 'martes', 'miércoles','jueves','viernes','sábado'],
                    value='lunes',
                    description='Día de la semana:',
                    disabled=False,
                    layout=widgets.Layout(width='250px'))
# definir dia a seleccionar del dataframe
SELECCION_DIA_CLUSTER=widgets.Select(
                    options=['lunes', 'martes', 'miércoles','jueves','viernes','sábado'],
                    value='lunes',
                    description='Día de la semana:',
                    disabled=False,
                    layout=widgets.Layout(width='250px'))

SELECCION_TIPO_CLUSTER=widgets.Select(
                    options=['KMeans', 'GaussianMixture'],
                    value='KMeans',
                    description='Tipo cluster:',
                    disabled=False,
                    layout=widgets.Layout(width='250px'))

SELECCION_NUM_CLUSTER=widgets.Select(
                    options=[f'Cluster {i}' for i in range(23)],
                    value='Cluster 0',
                    description='N° Cluster:',
                    disabled=False,
                    layout=widgets.Layout(width='250px'))

#  ----------------------------------------------------------------------------
#        DEFINIR PANTALLAS GRAFICAS
# ----------------------------------------------------------------------------
PANTALLA_GRAFICA_INICIO = widgets.Output(layout = widgets.Layout(height='800px',width='800px',overflow='auto'))
PANTALLA_GRAFICA_PLOT_RAWMAP = widgets.Output(layout = widgets.Layout(height='800px',width='800px',overflow='auto'))
PANTALLA_GRAFICA_CLUSTER = widgets.Output(layout = widgets.Layout(height='800px',width='800px',overflow='auto'))
PANTALLA_GRAFICA_OPTIMIZACION = widgets.Output(layout = widgets.Layout(height='800px',width='800px',overflow='auto'))
PANTALLA_GRAFICA_RESULTADOS = widgets.Output(layout = widgets.Layout(height='800px',width='800px',overflow='auto'))

#  ----------------------------------------------------------------------------
#        FUNCIONES INTERACTIVAS
# ----------------------------------------------------------------------------
def verificar_dia_cluster(change):
    global PANTALLA_GRAFICA_CLUSTER,ETIQUETA_DIA_CLUSTER,SELECCION_DIA_CLUSTER
    with PANTALLA_GRAFICA_CLUSTER:
        change.new=SELECCION_DIA_CLUSTER.value
        ETIQUETA_DIA_CLUSTER.value=f'<h4>Clusters para el día: {change.new.upper()}</h4>'

def resultados_dia_cluster(change):
    global PANTALLA_GRAFICA_OPTIMIZACION,ETIQUETA_RESULTADOS_OPTIMIZACION,n_cluster
    with PANTALLA_GRAFICA_OPTIMIZACION:
        dia_valor=SELECCION_DIA_CLUSTER.value
        change.new=SELECCION_NUM_CLUSTER.value
        ETIQUETA_RESULTADOS_OPTIMIZACION.value=f'<h4>Resultador del día {dia_valor.upper()} para el {change.new}</h4>'
#  ----------------------------------------------------------------------------
#        EVENTOS
# ----------------------------------------------------------------------------
def evento_cargar_dataframe(b):
    global PANTALLA_GRAFICA_INICIO,UPLOAD,df,df_cleaned
    PANTALLA_GRAFICA_INICIO.clear_output()
    with PANTALLA_GRAFICA_INICIO:
        # cargar dataframe
        # obtener el nombre del archivo
        input_file=io.BytesIO(list(UPLOAD.value.values())[0]['content'])

        # guardar en dataframe
        df=pd.read_excel(io=input_file,engine='openpyxl')
        
        # mostrar dataframe
        display(widgets.HTML('<h4>10 primeros registros del dataframe:</h4>'))
        display(pn.widgets.DataFrame(df.head(10), width=800,autosize_mode='fit_viewport'))
        display(widgets.HTML('<h4>Información de las variables:</h4>'))
        display(df.info())
        
        # preprocesar datos
        df_selected=escoger_variables(df,['Día','Interlocut','Cliente','Latitud','Longitud','Pedido','Real','T.Ruta'])
        df_cleaned=limpiar_dataframe(df_selected)
        display(widgets.HTML('<h4>Data preprocesada:</h4>'))
        display(df_cleaned)# mostrar data preprocesada

def evento_plotear_rawmap(b):
    global PANTALLA_GRAFICA_PLOT_RAWMAP,df_cleaned
    PANTALLA_GRAFICA_PLOT_RAWMAP.clear_output()
    with PANTALLA_GRAFICA_PLOT_RAWMAP:
        # leer dia seleccionado
        dia=SELECCION_DIA.value
        
        # establecer data del dia
        df_day=df_cleaned.loc[df_cleaned['Día']==dia].copy().reset_index(drop=True)
        display(df_day.head())
        
        # plotear data del dia
        plot_raw_map(df_day,dia)

def evento_plotear_cluster(b):
    global PANTALLA_GRAFICA_CLUSTER,df_cleaned,df_export,dia
    PANTALLA_GRAFICA_CLUSTER.clear_output()
    with PANTALLA_GRAFICA_CLUSTER:
        # leer dia seleccionado
        dia=SELECCION_DIA_CLUSTER.value
        
        # leer tipo de clustering
        tipo_cluster=SELECCION_TIPO_CLUSTER.value
        
        # diccionario de clustering
        dicc_clustering={'KMeans':KMeans,'GaussianMixture':GaussianMixture}
        
        # preparar data del dia
        df_day=df_cleaned.loc[df_cleaned['Día']==dia].copy().reset_index(drop=True)
        
        # preparar data escalada
        df_done=escalar_variables(df_day)
        
        # entrenar cluster acorde a selección
        df_export=entrenar_cluster(df_done,df_day,dicc_clustering[tipo_cluster],23)
        
        # plotear clusters
        plot_clusters(df_export)
        
def evento_plotear_rutas(b):
    global PANTALLA_GRAFICA_OPTIMIZACION,df_export,n_cluster,df_test,arcos_sorted
    PANTALLA_GRAFICA_OPTIMIZACION.clear_output()
    with PANTALLA_GRAFICA_OPTIMIZACION:
        # extraer numero del cluster
        n_cluster=int(SELECCION_NUM_CLUSTER.value.split()[1])
        
        # dataframe del cluster elegido
        df_test=df_export.loc[df_export.Cluster==n_cluster].copy().reset_index(drop=True)
        
        # establecer coordenadas fijas de la planta de ATE
        centro=(-12.066883514538135, -76.97871779045312)
        coor_lat=np.insert(df_test.Latitud.values,0,centro[0],axis=0)# insertar la latitud del centro
        coor_lon=np.insert(df_test.Longitud.values,0,centro[1],axis=0)# insertar la longitud del centro
        
        # mostrar dataframe a prueba
        display(df_test.head())
        try:
            # optener la mejor ruta para el cluster n
            status,arcos_activos,arcos_sorted=ruteo_dinamico(df_test,800)
            
            # obtener status
            status=str(status).split(sep='.')[1]
            dicc_status={'FEASIBLE_SOLUTION':'FACTIBLE','OPTIMAL_SOLUTION':'ÓPTIMA'}
            display(widgets.HTML(f'''<h4>La solución es de tipo: {dicc_status[status]}</h4>'''))
            
            # plotear la mejor ruta acorde al cluster
            plot_mejor_ruta(coor_lat,coor_lon,arcos_sorted,n_cluster,zoom=12)
            
            # 
        except:
            print('El cluster excede la cantidad de variables aceptadas por la versión académica de Cplex IBM')

def evento_mostrar_resultados(b):
    global PANTALLA_GRAFICA_RESULTADOS,df_test,arcos_sorted,df_result
    PANTALLA_GRAFICA_RESULTADOS.clear_output()
    with PANTALLA_GRAFICA_RESULTADOS:
        # llamar a la función para mostrar los resultados del enrutamiento
        df_result=presentar_ruta(df_test,arcos_sorted)
        # mostrar resultados
        display(widgets.HTML('<h4>Rutas ordenadas de la planta</h4>'))
        display(df_result)
        display(BOTON_GUARDAR_RESULTADOS)

def evento_exportar_resultados(b):
    global df_result
    guardar_ruta(df_result)
    
#  ----------------------------------------------------------------------------
#        ASIGNACIÓN DE EVENTOS
# ----------------------------------------------------------------------------
BOTON_MOSTRAR_DATOS.on_click(evento_cargar_dataframe)
BOTON_PLOTEAR_MAPA.on_click(evento_plotear_rawmap)
BOTON_PLOTEAR_CLUSTER.on_click(evento_plotear_cluster)
BOTON_PLOTEAR_CLUSTER.on_click(verificar_dia_cluster)
BOTON_PLOTEAR_RUTAS.on_click(evento_plotear_rutas)
BOTON_PLOTEAR_RUTAS.on_click(resultados_dia_cluster)
BOTON_PLOTEAR_RUTAS.on_click(evento_mostrar_resultados)
BOTON_GUARDAR_RESULTADOS.on_click(evento_exportar_resultados)
# ----------------------------------------------------------------------------
#         Funciones para Construir Componentes de Interfaz Gráfica
# ----------------------------------------------------------------------------
# Construcción de pestañas
def contruir_pestana_inicio():
    contenido_instrucciones=widgets.Box([INSTRUCCIONES_DE_PESTANA_INICIO])
    contenido_botones=widgets.HBox([UPLOAD,BOTON_MOSTRAR_DATOS])
    contenido_dataframe=widgets.Box([PANTALLA_GRAFICA_INICIO])
    contenido_total=widgets.VBox([contenido_instrucciones,contenido_botones,contenido_dataframe])
    return contenido_total

def contruir_pestana_rawmap():
    contenido_instrucciones=widgets.Box([INSTRUCCIONES_DE_RAWMAP])
    contenido_plotear=widgets.HBox([SELECCION_DIA,BOTON_PLOTEAR_MAPA])
    contenido_raw_map=widgets.Box([PANTALLA_GRAFICA_PLOT_RAWMAP])
    contenido_total=widgets.VBox([contenido_instrucciones,contenido_plotear,contenido_raw_map])
    return contenido_total

def construir_pestana_cluster():
    contenido_instrucciones=widgets.Box([INSTRUCCIONES_DE_CLUSTERS])
    contenido_plotear=widgets.HBox([SELECCION_DIA_CLUSTER,SELECCION_TIPO_CLUSTER,BOTON_PLOTEAR_CLUSTER])
    contenido_cluster=widgets.Box([PANTALLA_GRAFICA_CLUSTER])
    contenido_total=widgets.VBox([contenido_instrucciones,contenido_plotear,contenido_cluster])
    return contenido_total

def construir_pestana_optimizacion():
    contenido_instrucciones=widgets.Box([INSTRUCCIONES_DE_OPTIMIZACION])
    contenido_dia_cluster=widgets.Box([ETIQUETA_DIA_CLUSTER])
    contenido_botones_cluster=widgets.HBox([SELECCION_NUM_CLUSTER,BOTON_PLOTEAR_RUTAS])
    contenido_optimizacion=widgets.Box([PANTALLA_GRAFICA_OPTIMIZACION])
    contenido_total=widgets.VBox([contenido_instrucciones,contenido_dia_cluster,contenido_botones_cluster,
                                  contenido_optimizacion])
    return contenido_total

def contruir_pestana_resultados():
    contenido_etiqueta=widgets.Box([ETIQUETA_RESULTADOS_OPTIMIZACION])
    contenido_resultado=widgets.Box([PANTALLA_GRAFICA_RESULTADOS])
    contenido_total=widgets.VBox([contenido_etiqueta,contenido_resultado])
    return contenido_total

def construir_pestanas():
    # crear tablero
    pestanas_programa=widgets.Tab()
    # crear contenido
    pestanas_programa.children=[contruir_pestana_inicio(),contruir_pestana_rawmap(),construir_pestana_cluster(),
                               construir_pestana_optimizacion(),contruir_pestana_resultados()]
    # asignar titulos a pestañas
    titles=['INICIO','MAPA','CLUSTERS','OPTIMIZACIÓN','RESULTADOS']
    for i,title in enumerate(titles):
        pestanas_programa.set_title(i,title)
    # retornar pestaña padre
    return pestanas_programa

def app():
    display(widgets.Box([construir_pestanas()]))
    
# llamar app
app()

Box(children=(Tab(children=(VBox(children=(Box(children=(HTML(value='\n    <h2>Proyeyecto de Python Fundamenta…